# Progress Report


**Group Name**: support nectar machines

**Team members**: David Hofer (Cyber Security), Frederieke Lohmann (Data Science), Arvid Ban (Computer Science), Yi-Yi Ly (Neuroinformatics)

## 1. Introduction

In our solution for the UBS Challenge at Datathon 2024, we pursue a diverse approach of modeling brand interaction data on social media. Our aim is to provide investors with a broad information base and sophisticated insights based on different methods using the data provided by the UBS Evidence Lab which tracks the popularity of brands on Instagram. Examples of the data collected are for example number of followers, pictures, videos, comments and likes.

Our methods focus both on identifying viral brands and outliers based on various features and embeddings, and on predictive model of performance metrics useful for investors.

## 2. Data Processing

### Exploratory Data Analysis

The dataset consists of 704'313 rows from  **706 brands** that were recorded from 2015-01-03 to 2023-09-16. The brands are grouped into 20 main competitive sets that vary in size from 1 brands to 164 brands. The dataset contains information such as legal entity and stock exchange name that could be used to augment the dataset with financial information, as well as the actual interaction data in the form of likes, followers, comments, pictures and videos.

![Correlation matrix](corrmatrix.png)


The figure above displays the correlations between the main numerical features. It shows for example that pictures are highly correlated with likes and comments, more than e.g. videos.

### Data cleaning and preprocessing

In our preprocessing, we removed a large number of features that we deemed generally uninformative, as well as the two constant features `period` and `calculation_type`. Our primary focus was on modeling the social media user interaction, thus columns like `legal_entity_name`, `ultimate_parent_legal_entity_name`, `primary_exchange_name` did not provide any additional insights to us. Furthermore, there were lots of duplicate data rows with the only difference being the `compset` value, leading us to reject the feature as it did not seem to be informative. And finally, as our investigation of the `compset_group` feature found that most groups only contain a single or few brands and is thus not suitable for comparative analysis between in-group brands, we also left it out. We 
We experimented both with and without the `domicile_country_name` (which we previously cleaned by replacing certain noisy strings), but as it did not lead to a significant improvement of our methods, we opted towards simplicity and left it out too.
We also assumed that many of these aforementioned features are anyway strongly correlated among each other and with the brand, leading to dimishing returns in including them.

In the next step, we completely removed brands with too much missing data in one of the five numerical features `brand`, `followers`, `pictures`, `videos`, `comments`, and `likes`, based on a 70% threshold.

We also standardized all numerical features by subtracting the mean and dividing by the standard deviation. We then imputed the remaining values in the time series by using the forward-fill method, with the intention of not leaking future data. In the few cases where an initial datapoint was missing, we imputed it using the first value occuring in the sequence.

Lastly, we also created a train-test split for evaluation. For this, we grouped all the time series by brand, and for each brand cut off the last 20% of the datapoints as a test set.


Our philosophy was to largely focus on the raw features and let our models learn directly from them. However, we experimented with various engineered advanced features, such as rolling averages, exponential moving averages, growth rates, rolling minimum, maximum, and standard deviation, as well as time lag features (older features from previous timesteps shifted forward). In the end, we decided to augment our features with the rolling min, max, and std as well as time lag features.

We also came up with custom user brand engagement metrics, namely the `engagement_rate_per_post`, which is calculated as $erpp = \frac{likes + comments}{followers * (pictures + videos)}$. This metric counts the number of interactions on a weekly basis with the brand, normalized by the frequency of posts as well as the size of the user base.





## 3. Modeling Approach

We pursued two different approaches for modeling the trends in the social media interaction dataset. 
Both are based on our custom metric, `engagement_rate_per_post`.

### Peak detection

The first approach uses `engagement_rate_per_post` as a surrogate for a brand's performance on social media, and therefore also as an indicator of emerging trends.
We aim to identify spikes in the metric of the brand over time, which indicate that the given brand is currently subject of a social media trend, and therefore potentially interesting to stakeholders.
Since a single spike might not indicate general trend potential of the brand, we use the 
peak rate $pr = \frac{\# \text{peaks}}{\# \text{weeks}} $  of a brand for identifying outliers amongst the brands. Only the datapoints not in the qth quantile of the peak rate distribution are considered anomalies and therefore interesting.

This parametrization via the quantile allows stakeholders to exert tight control over the precise number of outliers they wish to identify. The larger the quantile, the fewer outliers will be retrieved and the more extreme those retrieved will be.

The peaks are extracted using scipy's peak detection algorithm with a threshold at 30% of the maximum `engagement_rate_per_post` of the given brand.

\
\
**Algorithm**

The outlier detection algorithm works as follows:
```
q = 98
pr = []

for each brand:
    t <- 0.3 * max(engagement_rate_time_series)
    pr.append(calculate_peak_rate(engagement_rate_time_series, t))

outliers <- complement(quantile(pr, q))

return outliers
```
\
\
**Results**



![](peak_detect_plots.png)


### LSTM prediction task and embedding
The second approach focused on using a RNN, specifically a LSTM, to model the data. RNNs are suitable for this kind of time series data, as they are able to aggregate and remember an internal state over multiple time steps and can use this state for further downstream tasks. We used this in two ways. First, we trained our LSTM in a supervised way to predict our engagement metric `engagement_rate_per_post` 4 weeks in the future, after seeing 10 weeks of features (10 timesteps). Given a contiguous time series for a specific brand, this time series is split into individual samples with distinct start and end sequence dates, always predicting the metric 4 weeks ahead of the last sequence datapoint. The idea here is that if the model is able to accurately solve this task, it can be used to predict user engagement with a brand in one month worth of time, based on the recent history and rather simple features based on it, thus modelling and predicting the future engagement with the brand. By analyzing specific brands, one can identify opportunities or threats by detecting changes in engagement early. 

Our predictive model consists of a 2-layer LSTM with hidden size 64. The hidden state of the last input feature is fed into a two-layer MLP, with hidden size 64, ReLU activation and a single output node. The model is then trained on input sequences of length 10 with label `engagement_rate_per_post` derived from the features in 4 weeks time. Hyperparameters and training setup used are batch size 64, learning rate 0.001, Adam optimizer, and 20 epochs training with MSE loss.
We evaluated the performance of our model on a test set and compared to the performance of our baseline, a Linear Regression model fitted to all the last datapoints in the training sequences. On the test set, the LSTM had a MSE of 0.78, compared to the MSE of the Linear Regression model of 1.21.

Another modeling approach we pursued is a Gaussian Process, which allows to estimate uncertainty of future predictions. It proved to be an accurate method for current data, but as data becomes more complex and sophisticated it remains to be seen which model is more viable.

![LSTM architecture](architecture.jpeg)

The second application for our LSTM is due to the fact that we can understand the internal state of the model as an embedding of the recent brand history in terms of user interaction and posting activity. Using this embedding, obtained by feeding in features from $n$ consecutive datapoints and extracting the hidden state afterwards, can be used in a multitude of ways. 

### Anomaly detection on LSTM embedding 

**Anomaly detection**

As soon as we have an embedding of each brand already encodes the time in a single data point, we can then run classical anomaly detection algorithms on the time-independent latent space.

We considered the following methods:
* Isolation Forest
* K-Means clustering
* Local Outlier Factor

Since local outlier factor requires careful tuning of the neighborhood size, we decided against using this method.
For K-Means clustering, the model assumption is that the clusters are spherical. We used silhouette score to tune the number of clusters and found that this does not provide conclusive results for the optimal cluster size on the embeddings produced by the LSTM. We assume that this is due to the non-spherical geometric structure that we can also see when visually inspecting the latent space.

Thus, we rely on Isolation forest as our anomaly detection method, since it does not make any assumptions about the geometric structure and is rather stable with respect to its hyperparameter, the number of estimators used.
The results of predicting outliers in the latent space of the LSTM can be seen in the following PCA plot of the LSTM embedding. Blue indicate outliers and red indicate inliers as classified by the isolation forest.

![](./latent_if.png)
\
\
\
As of now, we've not been able to conclusively identify which factors of variation exactly the latent space encodes, but we were able to qualitatively verify that they neither correspond to the brand labelling nor the time point of the given test subsequence with respect relative to other test subsequences of the same brand.
We concludes this from the seemingly random distribution of these features in latent space.

![](./brands_latent.png)  ![](./time_latent.png)



To illustrate, we plotted the time series of the engagement rate per post and follower and their significant peaks of four brands. The y axis is normalized. We can thus see that Star Wars has a higher engagement rate than the other brands, but the peaks are more spread.
For Roger Vivier for example, we can see that even though there is a pattern of significant peaks, the general trend of the peaks is descending.


We thus hypothesize that these latent embeddings encode some relevant degrees of freedom in the input features.
The outliers found with the isolation forest would then correspond to outliers based on these degrees of freedom.
Further analyses have to be conducted to quantify with respect to what features the identified datapoints can be considered outliers, and whether this is a useful proxy for brand relevance.









### Model enhancement

One big avenue for further investigation is the analysis of the embeddings computed by our LSTM. This approach requires more detailed evaluation of the information encoded in the embeddings, as well as methods for clustering the embeddings and detecting outliers and trends based on the embeddings. 

This would also benefit from a more sophisticated performance metric for brands on social media, potentially augmented by external data. Such an improved performance metric would force the LSTM model to learn and extract representative and insigthful features from the input data and embed them in a latent space during its prediction task in order to perform well.

Insightful performance metrics could also help in evaluating brands identified by different methods such as outlier and trend detection through clustering or the predictive modelling approach.

The prediction task would also benefit from investments in hyperparameter tuning and experimenting with the model architecture, as we only had limited time to investigate this. A competitive alternative in terms of architecture to LSTMs are GRUs, allowing for comparison between the two RNN types.

### Outlook

We think that our approach presents a promising direction for modeling and understanding brand trends on social media. It allows investors to predict user engagement ahead of time, one month in advance, enables them to identify brands going viral based on identifying spikes, and opens the door for interesting further research in the direction of brand history embeddings used for anomaly detection. These approaches could be greatly enhanced by using past data for peak detection, allowing for identification of virality and trends across brands and sectors.

We think that in this dynamic area, a diversified modeling approach is crucial to guarantee the edge in identifying winning brands, and tried to show such an approach in our project.

An improved pipeline would integrate the existing modeling approaches in a recommender system, which would directly rank brands based on investor preferences such as risk-averseness.